In [2]:
import pandas as pd
pd.options.display.max_rows=100
pd.options.display.max_columns=100

from sportsreference.ncaab.roster import Roster
from sportsreference.ncaab.roster import Player


In [24]:
#pulling player data into one dataframe

team = Roster('ARIZONA-STATE') #get team roster object
team_players=[]
for player in team.players:
    team_players.append(player.player_id) #add team player_ids to a list
    
player_dfs=[] #eventually store player dataframes
for name in team_players:
    player=Player(name)
    #use try catch because some players have null values 
    try:
        test=player.field_goal_attempts
        player_df=player.dataframe
        player_df['name']=player.name #add name column
        player_dfs.append(player_df)
    except:
        continue 
        
player_data=pd.concat(player_dfs) #concaatenate all player dfs into one  

player_data['year']=player_data.index  #add year column, which was previously the index
player_data=player_data.reset_index(drop=True)

#for some reason, year is added as a tuple. strip the relevant year string
for row in range(player_data.shape[0]):
    player_data.at[row,'year']=player_data.at[row,'year'][0]

#reorder columns (put name and year first)
cols=player_data.columns.to_list()
player_data=player_data[cols[-2:]+cols[:-2]]

player_data

,name,year,assist_percentage,assists,block_percentage,blocks,box_plus_minus,conference,defensive_box_plus_minus,defensive_rebound_percentage,defensive_rebounds,defensive_win_shares,effective_field_goal_percentage,field_goal_attempts,field_goal_percentage,field_goals,free_throw_attempt_rate,free_throw_attempts,free_throw_percentage,free_throws,games_played,games_started,height,minutes_played,offensive_box_plus_minus,offensive_rebound_percentage,offensive_rebounds,offensive_win_shares,personal_fouls,player_efficiency_rating,player_id,points,points_produced,position,steal_percentage,steals,team_abbreviation,three_point_attempt_rate,three_point_attempts,three_point_percentage,three_pointers,total_rebound_percentage,total_rebounds,true_shooting_percentage,turnover_percentage,turnovers,two_point_attempts,two_point_percentage,two_pointers,usage_percentage,weight,win_shares,win_shares_per_40_minutes
0,Remy Martin,2017-18,23.5,94.0,0.3,2.0,4.8,pac-12,1.3,10.9,73.0,0.9,0.502,236.0,0.453,107.0,0.398,94.0,0.755,71.0,32.0,1.0,6-0,762.0,3.5,3.4,22.0,1.7,48.0,18.8,remy-martin-2,308.0,328.0,Guard,2.7,36.0,arizona-state,0.263,62.0,0.371,23.0,7.2,95.0,0.549,13.3,43.0,174.0,0.483,84.0,21.5,170,2.7,0.141
1,Remy Martin,2018-19,29.6,160.0,0.1,1.0,3.8,pac-12,0.5,8.2,80.0,1.3,0.464,346.0,0.402,139.0,0.361,125.0,0.736,92.0,32.0,28.0,6-0,1043.0,3.3,2.2,21.0,2.0,43.0,16.6,remy-martin-2,413.0,449.0,Guard,2.1,40.0,arizona-state,0.399,138.0,0.312,43.0,5.2,101.0,0.509,14.2,67.0,208.0,0.462,96.0,21.6,170,3.3,0.128
2,Remy Martin,2019-20,28.3,14.0,1.0,1.0,13.5,pac-12,6.3,14.1,14.0,0.4,0.585,41.0,0.512,21.0,0.293,12.0,0.75,9.0,3.0,3.0,6-0,95.0,7.3,0.0,0.0,0.3,1.0,27.3,remy-martin-2,57.0,54.0,Guard,5.6,11.0,arizona-state,0.341,14.0,0.429,6.0,7.0,14.0,0.61,14.6,8.0,27.0,0.556,15.0,24.7,170,0.7,0.292
3,Remy Martin,Career,27.1,268.0,0.2,4.0,4.7,,1.1,9.6,167.0,2.6,0.486,623.0,0.429,267.0,0.371,231.0,0.745,172.0,67.0,32.0,6-0,1900.0,3.6,2.6,43.0,4.1,92.0,18.0,remy-martin-2,778.0,830.0,Guard,2.5,87.0,arizona-state,0.343,214.0,0.336,72.0,6.1,210.0,0.531,13.9,118.0,409.0,0.477,195.0,21.7,170,6.7,0.141
4,Rob Edwards,2015-16,11.1,46.0,0.1,1.0,-3.3,horizon,-0.5,12.7,104.0,1.3,0.467,331.0,0.393,130.0,0.36,119.0,0.748,89.0,32.0,29.0,6-4,994.0,-2.8,2.4,21.0,-0.2,71.0,11.3,rob-edwards-1,398.0,366.0,Guard,2.3,37.0,arizona-state,0.375,124.0,0.395,49.0,7.4,125.0,0.514,22,109.0,207.0,0.391,81.0,26.1,200,1.0,0.041
5,Rob Edwards,2016-17,13.2,53.0,0.6,5.0,0.2,horizon,-0.6,14.2,110.0,1.3,0.491,404.0,0.413,167.0,0.369,149.0,0.765,114.0,31.0,30.0,6-4,964.0,0.9,3.2,29.0,1.2,68.0,18.0,rob-edwards-1,511.0,470.0,Guard,2.5,41.0,arizona-state,0.441,178.0,0.354,63.0,8.3,139.0,0.538,16.7,95.0,226.0,0.46,104.0,29.8,200,2.5,0.102
6,Rob Edwards,2017-18,NaN,NaN,NaN,NaN,NaN,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6-4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,rob-edwards-1,NaN,NaN,Guard,NaN,NaN,arizona-state,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,200,NaN,NaN
7,Rob Edwards,2018-19,8.2,36.0,0.7,5.0,1.6,pac-12,1.0,10.8,83.0,1.1,0.438,273.0,0.344,94.0,0.311,85.0,0.835,71.0,28.0,21.0,6-4,820.0,0.6,1.2,9.0,0.9,55.0,11.8,rob-edwards-1,310.0,281.0,Guard,1.9,28.0,arizona-state,0.498,136.0,0.375,51.0,6.0,92.0,0.495,10.6,37.0,137.0,0.314,43.0,20.4,200,2.0,0.096
8,Rob Edwards,2019-20,2.4,1.0,0.0,0.0,3.3,pac-12,-1.2,7.7,6.0,0.2,0.594,32.0,0.438,14.0,0.344,11.0,0.909,10.0,3.0,3.0,6-4,75.0,4.5,2.5,2.0,0.2,6.0,17.4,rob-edwards-1,48.0,39.0,Guard,0.6,1.0,arizona-state,0.625,20.0,0.5,10.0,5.1,8.0,0.645,15.8,7.0,12.0,0.333,4.0,25.3,200,0.4,0.221
9,Rob Edwards,Career,10.7,136.0,0.5,11.0,-0.5,,-0.1,12.5,303.0,3.9,0.473,1040.0,0.389,405.0,0.35,364.0,0.78,284.0,94.0,83.0,6-4,2853.0,-0.4,2.3,61.0,2.0,200.0,13.8,rob-edwards-1,1267.0,1156.0,Guard,2.2,107.0,arizona-state,0.44,458.0,0.378,173.0,7.3,364.0,0.522,17,248.0,582.0,0.399,232.0,25.7,200,5.9,0.082


In [25]:
#defining functions to create data for report 

def who_to_foul(df,year): #enter year as string (example :'2018-19')
    df=df.query('year==@year') #filter by year
    df=df.query('minutes_played/games_played > 8') #filter players with more than 8 mpg
    df=df.sort_values(by=['free_throw_percentage'],ascending=True)
    return_df=df[['name','free_throw_percentage']]
    return(return_df)

def who_draws_fouls(df,year):
    df=df.query('year==@year') #filter by year
    df=df.query('minutes_played/games_played > 8') #filter players with more than 8 mpg
    
    df=df.reset_index()
    df['fta_per_40']=''
    for row in range(df.shape[0]):
        df.at[row,'fta_per_40']=40*(df.at[row,'free_throw_attempts']/df.at[row,'minutes_played'])
    
    df=df.sort_values(by=['fta_per_40'],ascending=False)
    return_df=df[['name','fta_per_40']]
    return(return_df)

def three_shooters(df,year):
    df=df.query('year==@year') #filter by year
    df=df.query('minutes_played/games_played > 8') #filter players with more than 8 mpg
    
    df=df.reset_index()
    df['three_att_per_game']=''
    for row in range(df.shape[0]):
        df.at[row,'three_att_per_game']=df.at[row,'three_point_attempts']/df.at[row,'games_played']
    
    df=df.sort_values(by=['three_point_percentage'],ascending=False)
    return_df=df[['name','three_att_per_game','three_point_percentage']]
    return(return_df[0:4])

def who_turns_ball_over(df,year):
    df=df.query('year==@year') #filter by year
    df=df.query('minutes_played/games_played > 8') #filter players with more than 8 mpg
    df=df.sort_values(by=['turnover_percentage'],ascending=False)
    return_df=df[['name','turnover_percentage']]
    return(return_df)

def who_grabs_offensive_rebounds(df,year):
    df=df.query('year==@year') #filter by year
    df=df.query('minutes_played/games_played > 8') #filter players with more than 8 mpg
    
    df=df.reset_index()
    df['oreb_per_game']=''
    for row in range(df.shape[0]):
        df.at[row,'oreb_per_game']=df.at[row,'offensive_rebounds']/df.at[row,'games_played']
    
    df=df.sort_values(by=['oreb_per_game'],ascending=False)
    return_df=df[['name','oreb_per_game']]
    return(return_df[0:4])
    


who_grabs_offensive_rebounds(player_data,'2018-19')
    

,name,oreb_per_game
3,Romello White,2.26471
2,Kimani Lawrence,1.05882
5,Mickey Mitchell,1
0,Remy Martin,0.65625
